# Spatial Analyses script

This script uses the resuilts from the temporally-specific analyses (aka the results from "TemporallySpecificAnalyses_SVC_mainFunctions" to uncover the spatial patterns underlying each significant cluster.

This script is responsible for the spatial analyses and figures in the results, as well as the related supplemental figures.

### import modules

In [1]:
%reload_ext autoreload
%autoreload 2
%run EEG_auxiliary_module_sptm_wICA.ipynb
import matplotlib.pyplot as plt
#import statsmodels.stats.multitest.multipletests
import numpy as np
import pandas as pd
import os
import mne
from IPython.core.debugger import set_trace
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
display(HTML("<style>div.output_scroll { height: 40em; }</style>"))
#disable sklearn warnings
import sys
from scipy import signal as signal
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    os.environ['PYTHONWARNINGS']='ignore'
    
from scipy.spatial.distance import  squareform
from statsmodels.stats.multitest import multipletests

%matplotlib qt


C:\Users\bachmanm\AppData\Local\Temp\ipykernel_16300\1491564398.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### find the group-averaged model coefs per stimulus in a defined time range

This will output the average and individual topomaps for a certain condition and certain timerange.

It will also output both the classifier filters and patterns. The latter is only used because it is more neurophysiologically interpretable.

Adjust lines 80 and 81 in the cell below to point to the output from "TemporallySpecificAnalyses_SVC_mainFunctions.""

In [13]:
def get_main_weights_per_stim (cond_main,event_main, time1, time2):
#     set_trace()
    subjects = [str(s) for s in [150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,166,167,168,169,170,171,173,174,176,177,178,180,181,182,183,185,186,188,189,190,191,192,193,194,195]]
    multiclass  = 'binary'#['binary','multi']
    
    
    coefsFile        = os.path.join('GroupResults','guggenmos_noJitFix','groupCoefs'+event_main+cond_main+str(time1)+'-'+str(time2)+'_v2_reref.npy')
    coefsFile_scaled = os.path.join('GroupResults','guggenmos_noJitFix','groupCoefs_scaled'+event_main+cond_main+str(time1)+'-'+str(time2)+'_v2_reref.npy')

#     %matplotlib qt

    if event_main == 'Stimulus':
        times = np.linspace(-200,1000,120)
    elif event_main == 'Dot':
        times = np.linspace(800,2000,120)
    elif event_main == 'Stimulus_long':
        times = np.linspace(-200,2000,220)
    
    condWord = cond_main+'s'
    numClasses = 5
    
    if cond_main == 'Response-0' or cond_main == 'Value_absMag' or cond_main == 'Value_binary':
        condWord = cond_main
        numClasses = 2
        
    if cond_main == 'Value_absMag_v2' or cond_main == 'Value_binary_v2':
        condWord = cond_main
        numClasses = 3
        
    if cond_main == 'Value-2':
        condWord = cond_main   
        numClasses = 5 # 2 classes only 1 set of coefs
    
    print(condWord)
    
    # This is fixes the z-limits for each plot. They are manually set based on the absolute maximum value for each condition.
    if cond_main == 'Face':
        vmaxAbs = .004 # For Faces
        vmaxAbs_sep = .004 
    elif cond_main =='Color': 
        vmaxAbs = .001 # For Colors
        vmaxAbs_sep = .003 
    elif cond_main == 'FaceValue':
        vmaxAbs = .005 # For FaceValues
        vmaxAbs_sep = .008
    elif cond_main =='ColorValue': 
        vmaxAbs = .004 # For ColorValues
        vmaxAbs_sep = 0.010
    elif cond_main =='Value-2': 
        vmaxAbs = .007 # For Value-2
        vmaxAbs_sep = 0.019
    elif cond_main =='Response-0': 
        vmaxAbs = .007 # For Response-0
        vmaxAbs_sep = .007 
    elif cond_main =='Value_absMag': 
        vmaxAbs = .006 # For Value_absMag
    elif cond_main =='Value_absMag_v2': 
        vmaxAbs = .003 # For Value_absMag_v2
    elif cond_main =='Value_binary': 
        vmaxAbs = .009 # For Value_binary
    elif cond_main =='Value_binary_v2': 
        vmaxAbs = .004 # For Value_binary_v2  
    
        
    t_ind1 = find_nearest_idx(time1,times)
    t_ind2 = find_nearest_idx(time2,times)

    eegData, _, _ = get_eeg_behav_data('102')
    infoStructure = eegData.info
    del eegData

    if os.path.exists(coefsFile):
        coefs_all        = np.load(coefsFile)
        coefs_all_scaled = np.load(coefsFile_scaled)
    else:
        coefs_all        = np.zeros((len(subjects),numClasses,64))
        coefs_all_scaled = np.zeros((len(subjects),numClasses,64))

        for s, subj in enumerate(subjects):
            coefs = np.load(os.path.join('Results',subj,'classification_guggenmos_noJitFix','coefs_'+condWord+'_'+event_main+'_10ms_v2_reref.npy'))
            coefs_scaled = np.load(os.path.join('Results',subj,'classification_guggenmos_noJitFix','coefs_scaled_'+condWord+'_'+event_main+'_10ms_v2_reref.npy'))
                        
            coefs        = np.nanmean(coefs[:,:,:,t_ind1:t_ind2+1],axis = -1)
            coefs_scaled = np.nanmean(coefs_scaled[:,:,:,t_ind1:t_ind2+1],axis = -1)
#             set_trace()
            for ch in range(coefs.shape[-1]):
                #get the confusion matrix for stimuli
                temp  = coefs[:,:,ch].copy()
                temp2 = coefs_scaled[:,:,ch].copy()
#                 set_trace()
                if numClasses > 2:
                    #Fill both sides of the diagonal to be able to extract the mean by np.nanmean
                    for i in range(1,temp.shape[0]):
                        for j in range(i):
                            temp[i,j]  = temp[j,i]
                            temp2[i,j] = temp2[j,i]                    
                
                coefs_all[s,:,ch]        = np.nanmean(temp, axis = 0).transpose()
                coefs_all_scaled[s,:,ch] = np.nanmean(temp2, axis = 0).transpose()
#         set_trace()
        np.save(coefsFile,coefs_all)
        np.save(coefsFile_scaled,coefs_all_scaled)
        
#     set_trace()
    coefMean        = np.mean(coefs_all,0)
    coefMean_scaled = np.mean(coefs_all_scaled,0)
    
    ### plot pattern for each stimulus
    if numClasses > 2:
        for ss in range(coefMean.shape[0]):
            fig, axes = plt.subplots(1,2)
            plt.subplot(1,2,1)
            plt.title('coefs', pad= 1)
            [title_txt, absmax] = plot_scalpmap(coefMean[ss,:], infoStructure,times[t_ind1],timeWindow=None)

            plt.subplot(1,2,2)
            plt.title('coefs_scaled', pad= 1)
            [title_txt,absmax] = plot_scalpmap(coefMean_scaled[ss,:], infoStructure,times[t_ind1],timeWindow=None,vrange=vmaxAbs_sep)


            figname = 'pattern' + event_main + cond_main + str(ss+1)+' ' + str(time1)+'-'+str(time2)+'ms_fixedRange_reref.eps'
            savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection_reref',figname)
            plt.suptitle(cond_main.replace('-','') + '-' + str(ss+1) + ' ' + str(time1)+'-'+str(time2)+'ms',x=.5, y=.85)
            plt.savefig(savefig,format='eps')

    ## plot averaged pattern
    fig, axes = plt.subplots()
    plt.subplot(1,2,1)
    plt.title('coefs', pad= 1)

    [title_txt,absmax] = plot_scalpmap(np.nanmean(coefMean,0), infoStructure,times[t_ind1],timeWindow=None)
    plt.subplot(1,2,2)
    plt.title('coefs_scaled', pad= 1)
    [title_txt,absmax] = plot_scalpmap(np.nanmean(coefMean_scaled,0), infoStructure,times[t_ind1],timeWindow=None,vrange=vmaxAbs)
   

    figname = 'pattern' + event_main + cond_main + 'Avg'+' ' + str(time1)+'-'+str(time2)+'ms_fixedRange_reref.eps'
    savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection_reref',figname)
    plt.suptitle(cond_main.replace('-','') + ' AvgStim' + ' ' + str(time1)+'-'+str(time2)+'ms',x=.5, y=.85)
    plt.savefig(savefig,format='eps')

    cond_main_Weights = coefMean
    cond_main_Patterns = coefs_all_scaled
    return cond_main_Weights, cond_main_Patterns

In [4]:
### Time ranges for significant decoding activity in the temporally-specific classifiers.

#Face: 90 to 1300
#Color: 80 to 650
#FaceV: 450 to 1410, 1560 to 1710
#ColorV: 100 to 190, 220 to 380, 450 to 660
#Value-2: 600 to 780, 990 to 1090, 1280 to 1410, 1430 to 1520, 1550 to 1680
# response 620 to 860, 970 to 1170, 1190 to 1510, 1810 to 1880
# 'Value_absMag: 1270 to 1430
# 'Value_absMag_v2: 110 to 210, 930 to 1010, 1030 to 1120, 1170 to 1830, 1850 to 1990
# 'Value_binary: 620 to 790, 870 to 1790
# 'Value_binary_v2: 310 to 390, 450 to 1910, 1930 to 1990

In [14]:
## Extract the average topos for each cluster in each condition.
cond_main_Weights,Face_Percp1_Patterns = get_main_weights_per_stim ('Face','Stimulus_long', 90, 1300)
cond_main_Weights,Face_Value1_Patterns = get_main_weights_per_stim ('FaceValue','Stimulus_long', 450, 1410)
cond_main_Weights,Face_Value2_Patterns = get_main_weights_per_stim ('FaceValue','Stimulus_long', 1560, 1710)
cond_main_Weights,Color_Percp1_Patterns = get_main_weights_per_stim ('Color','Stimulus_long', 80, 650)
cond_main_Weights,Color_Value1_Patterns = get_main_weights_per_stim ('ColorValue','Stimulus_long', 100,190)
cond_main_Weights,Color_Value2_Patterns = get_main_weights_per_stim ('ColorValue','Stimulus_long', 220,380)
cond_main_Weights,Color_Value3_Patterns = get_main_weights_per_stim ('ColorValue','Stimulus_long', 450,660)
cond_main_Weights,Overall_Value1_Patterns = get_main_weights_per_stim ('Value-2','Stimulus_long', 600,780)
cond_main_Weights,Overall_Value2_Patterns = get_main_weights_per_stim ('Value-2','Stimulus_long', 990,1090)
cond_main_Weights,Overall_Value3_Patterns = get_main_weights_per_stim ('Value-2','Stimulus_long', 1280,1410)
cond_main_Weights,Overall_Value4_Patterns = get_main_weights_per_stim ('Value-2','Stimulus_long', 1430,1520)
cond_main_Weights,Overall_Value5_Patterns = get_main_weights_per_stim ('Value-2','Stimulus_long', 1550,1680)
cond_main_Weights,Response1_Patterns = get_main_weights_per_stim ('Response-0','Stimulus_long', 620, 860)
cond_main_Weights,Response2_Patterns = get_main_weights_per_stim ('Response-0','Stimulus_long', 970, 1170)
cond_main_Weights,Response3_Patterns = get_main_weights_per_stim ('Response-0','Stimulus_long', 1190, 1510)
cond_main_Weights,Response4_Patterns = get_main_weights_per_stim ('Response-0','Stimulus_long', 1810, 1880)


Faces
0.010888659966738845
0.004
0.006987487617503405
0.004
0.00460460467337282
0.004
0.0072327713380831325
0.004
0.009307438946817563
0.004
0.005547114632823987
0.004
FaceValues
0.009516396390111054
0.008
0.008206849560185048
0.008
0.005526406765188402
0.008
0.005257058945741966
0.008
0.01123657093612144
0.008
0.007386505094778384
0.005
FaceValues
0.010862523306789652
0.008
0.005487362601334872
0.008
0.0044554544287267895
0.008
0.0060745039680624256
0.008
0.011183307877603462
0.008
0.0060782097176301465
0.005
Colors
0.012981261417722758
0.003
0.006217227755726066
0.003
0.005279215186192063
0.003
0.005766572753691905
0.003
0.009415957307672895
0.003
0.006084099187005314
0.001
ColorValues
0.013941043666192773
0.01
0.008949682111859413
0.01
0.006654537606684036
0.01
0.010113721042371421
0.01
0.010267166714582188
0.01
0.00870327288988475
0.004
ColorValues
0.012745354896669291
0.01
0.005918256071040294
0.01
0.005306863166737738
0.01
0.009023940985199748
0.01
0.013248756631856849
0.01
0.006

In [15]:
# Okay, now I'm going to 1) make a mean cluster across all types and clusters, correlate those with other categories, and test the overall difference.
import scipy
Face_Percep_Cluster = np.nanmean(Face_Percp1_Patterns,axis=1)
Face_Value_Cluster = (np.nanmean(Face_Value1_Patterns,axis=1) + np.nanmean(Face_Value2_Patterns,axis=1))/2
Color_Percep_Cluster = np.nanmean(Color_Percp1_Patterns,axis=1)
Color_Value_Cluster = (np.nanmean(Color_Value1_Patterns,axis=1) + np.nanmean(Color_Value2_Patterns,axis=1)+ np.nanmean(Color_Value3_Patterns,axis=1) )/3
Overall_Value_Cluster = (np.nanmean(Overall_Value1_Patterns,axis=1) + np.nanmean(Overall_Value2_Patterns,axis=1)+ np.nanmean(Overall_Value3_Patterns,axis=1)+ np.nanmean(Overall_Value4_Patterns,axis=1)+ np.nanmean(Overall_Value5_Patterns,axis=1))/5
Response_Cluster = (np.nanmean(Response1_Patterns,axis=1) + np.nanmean(Response2_Patterns,axis=1)+ np.nanmean(Response3_Patterns,axis=1) + np.nanmean(Response4_Patterns,axis=1))/4


FaceP_vs_FaceV = []
ColorP_vs_ColorV = []
FaceV_vs_ColorV = []
FaceV_vs_OverallV = []
ColorV_vs_OverallV = []
OverallV_vs_Response = []
FaceV_vs_Response = []
ColorV_vs_Response = []

# Pearson correlations between conditions.
for i in range(0,40):
    [a,b] = scipy.stats.pearsonr(Face_Percep_Cluster[i,:], Face_Value_Cluster[i,:])
    FaceP_vs_FaceV.append(a)
    [a,b] = scipy.stats.pearsonr(Color_Percep_Cluster[i,:], Color_Value_Cluster[i,:])
    ColorP_vs_ColorV.append(a)
    [a,b] = scipy.stats.pearsonr(Face_Value_Cluster[i,:], Color_Value_Cluster[i,:])
    FaceV_vs_ColorV.append(a)
    [a,b] = scipy.stats.pearsonr(Face_Value_Cluster[i,:], Overall_Value_Cluster[i,:])
    FaceV_vs_OverallV.append(a)
    [a,b] = scipy.stats.pearsonr(Color_Value_Cluster[i,:], Overall_Value_Cluster[i,:])
    ColorV_vs_OverallV.append(a)
    [a,b] = scipy.stats.pearsonr(Overall_Value_Cluster[i,:], Response_Cluster[i,:])
    OverallV_vs_Response.append(a)
    [a,b] = scipy.stats.pearsonr(Face_Value_Cluster[i,:], Response_Cluster[i,:])
    FaceV_vs_Response.append(a)
    [a,b] = scipy.stats.pearsonr(Color_Value_Cluster[i,:], Response_Cluster[i,:])
    ColorV_vs_Response.append(a)

# Z-transform the data.
FaceP_vs_FaceV_zscore = []
ColorP_vs_ColorV_zscore = []
FaceV_vs_ColorV_zscore = []
FaceV_vs_OverallV_zscore = []
ColorV_vs_OverallV_zscore  = []
OverallV_vs_Response_zscore  = []
FaceV_vs_Response_zscore = []
ColorV_vs_Response_zscore  = []
for i in range(0,40):
    FaceP_vs_FaceV_zscore.append(.5*np.log((1+FaceP_vs_FaceV[i])/(1-FaceP_vs_FaceV[i])))
    ColorP_vs_ColorV_zscore.append(.5*np.log((1+ColorP_vs_ColorV[i])/(1-ColorP_vs_ColorV[i])))
    FaceV_vs_ColorV_zscore.append(.5*np.log((1+FaceV_vs_ColorV[i])/(1-FaceV_vs_ColorV[i])))
    FaceV_vs_OverallV_zscore.append(.5*np.log((1+FaceV_vs_OverallV[i])/(1-FaceV_vs_OverallV[i])))
    ColorV_vs_OverallV_zscore.append(.5*np.log((1+ColorV_vs_OverallV[i])/(1-ColorV_vs_OverallV[i])))
    OverallV_vs_Response_zscore.append(.5*np.log((1+OverallV_vs_Response[i])/(1-OverallV_vs_Response[i])))
    FaceV_vs_Response_zscore.append(.5*np.log((1+FaceV_vs_Response[i])/(1-FaceV_vs_Response[i])))
    ColorV_vs_Response_zscore.append(.5*np.log((1+ColorV_vs_Response[i])/(1-ColorV_vs_Response[i])))


In [17]:
#One-sample ttest on z-scored data. 
print(scipy.stats.ttest_1samp(FaceP_vs_FaceV_zscore,popmean=0))
print(scipy.stats.ttest_1samp(ColorP_vs_ColorV_zscore,popmean=0))
print(scipy.stats.ttest_1samp(FaceV_vs_ColorV_zscore,popmean=0))
print(scipy.stats.ttest_1samp(FaceV_vs_OverallV_zscore,popmean=0))
print(scipy.stats.ttest_1samp(ColorV_vs_OverallV_zscore,popmean=0))
print(scipy.stats.ttest_1samp(OverallV_vs_Response_zscore,popmean=0))
print(scipy.stats.ttest_1samp(FaceV_vs_Response_zscore,popmean=0))
print(scipy.stats.ttest_1samp(ColorV_vs_Response_zscore,popmean=0))

TtestResult(statistic=-0.010534812238418683, pvalue=0.9916482980362076, df=39)
TtestResult(statistic=0.7062786811571646, pvalue=0.48421308506517524, df=39)
TtestResult(statistic=0.3076986622318719, pvalue=0.7599489329262533, df=39)
TtestResult(statistic=10.188189742359587, pvalue=1.5052183638571087e-12, df=39)
TtestResult(statistic=6.407528523059508, pvalue=1.408006237336219e-07, df=39)
TtestResult(statistic=10.999726051585997, pvalue=1.6094600148668403e-13, df=39)
TtestResult(statistic=7.321733961448829, pvalue=7.781456552102176e-09, df=39)
TtestResult(statistic=6.288211010171032, pvalue=2.0608778389558357e-07, df=39)


In [ ]:
# fdr correction
pvals = []
stat,pval = scipy.stats.ttest_1samp(FaceP_vs_FaceV_zscore,popmean=0)
pvals.append(pval)
stat,pval = scipy.stats.ttest_1samp(ColorP_vs_ColorV_zscore,popmean=0)
pvals.append(pval)
stat,pval = scipy.stats.ttest_1samp(FaceV_vs_ColorV_zscore,popmean=0)
pvals.append(pval)
stat,pval = scipy.stats.ttest_1samp(FaceV_vs_OverallV_zscore,popmean=0)
pvals.append(pval)
stat,pval = scipy.stats.ttest_1samp(ColorV_vs_OverallV_zscore,popmean=0)
pvals.append(pval)
stat,pval = scipy.stats.ttest_1samp(OverallV_vs_Response_zscore,popmean=0)
pvals.append(pval)

reject_by, p_values_corrected_by, _, _ = multipletests(pvals,alpha=.05,method='fdr_bh')

## Plotting the cluster-averaged topos (top of spatial figure)

Uncomment each line as needed.

In [ ]:
eegData, _, _ = get_eeg_behav_data('102')
infoStructure = eegData.info
#im, cn = mne.viz.plot_topomap(np.nanmean(Face_Percep_Cluster,axis=0),infoStructure, vlim=(-.003,.003),cmap='jet')
#im, cn = mne.viz.plot_topomap(np.nanmean(Color_Percep_Cluster,axis=0),infoStructure, vlim=(-.001,.001),cmap='jet')
#im, cn = mne.viz.plot_topomap(np.nanmean(Face_Value_Cluster,axis=0),infoStructure, vlim=(-.003,.003),cmap='jet')
#im, cn = mne.viz.plot_topomap(np.nanmean(Color_Value_Cluster,axis=0),infoStructure, vlim=(-.002,.002),cmap='jet')
#im, cn = mne.viz.plot_topomap(np.nanmean(Overall_Value_Cluster,axis=0),infoStructure, vlim=(-.006,.006),cmap='jet')
im, cn = mne.viz.plot_topomap(np.nanmean(Response_Cluster,axis=0),infoStructure, vlim=(-.004,.004),cmap='jet')



## Plotting the histograms (bottom of spatial figure).

Uncomment each collection of code as needed.

In [ ]:
#plt.hist(FaceP_vs_FaceV_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','FaceP_vs_FaceV_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

#plt.hist(ColorP_vs_ColorV_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','ColorP_vs_ColorV_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

#plt.hist(FaceV_vs_ColorV_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','FaceV_vs_ColorV_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

#plt.hist(FaceV_vs_OverallV_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','FaceV_vs_OverallV_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

#plt.hist(ColorV_vs_OverallV_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','ColorV_vs_OverallV_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

#plt.hist(OverallV_vs_Response_zscore,range = (-2.23,2.23))
#plt.axvline(x=0,c='black')
#plt.ylim((0,18))
#savefig = os.path.join('GroupResults','guggenmos_noJitFix','Figures','finalSelection','OverallV_vs_Response_zscore_reref.eps')
#plt.savefig(savefig,format='eps')

## Checking correlations between adjacent clusters. 

The distribution of these values are reported in the methods.

In [ ]:
# This tests for within-subject stability of clusters.
import scipy.stats
r_scores1 = [];
r_scores2 = [];
r_scores3 = [];
r_scores4 = [];

## Adjust this as necessary for your condition.
cluster1 = np.nanmean(Face_Value1_Patterns,axis=1)
cluster2 = np.nanmean(Face_Value2_Patterns,axis=1)
#cluster3 = np.nanmean(Color_Value3_Patterns,axis=1)
#cluster4 = np.nanmean(Overall_Value4_Patterns,axis=1)
#cluster5 = np.nanmean(Overall_Value5_Patterns,axis=1)

for i in range (0,40):
    [a,b] = scipy.stats.pearsonr(cluster1[i,:], cluster2[i,:])
    r_scores1.append(a)
    #[a,b] = scipy.stats.pearsonr(cluster2[i,:], cluster3[i,:])
    #r_scores2.append(a)
    #[a,b] = scipy.stats.pearsonr(cluster3[i,:], cluster4[i,:])
    #r_scores3.append(a)
    #[a,b] = scipy.stats.pearsonr(cluster4[i,:], cluster5[i,:])
    #r_scores4.append(a)
#fig, ax = plt.subplots()
#img,_ = mne.viz.plot_topomap(r_scores,infoStructure,cmap='jet',axes=ax)
#cbar = plt.colorbar(ax=ax,shrink=.75,orientation="vertical",mappable=img)
print(np.mean(r_scores1))
print(np.mean(r_scores2))
print(np.mean(r_scores3))
print(np.mean(r_scores4))